# Step Three: 2. Check-GPT-Output

## From customized XML/HTML to customized key-value pairs
#### 1. Run Table-Processing section powered GPT (Fill in your own OpenAI key and base_url, prompts, examples dicectly in code; Fill in examples for GPT in CSV)
#### 2. Check GPT output Tables.
#### 3. Sort, Clean and Integrate GPT-processed Tables (Remove File Name Folder, Table-Clean and integrate table1, table2, table3 ... into a JSON);
#### Note: Examples for E are saved in FewShot-XML.csv, and examples for RSC and SN are saved in FewShot-HTML.csv
#### Each customized key-value pairs are divided by "###", such as: 

In [14]:
"""###<Title of the Table><Corresponding Row Header><Corresponding Column Header>=<Entry Value>###"""

'###<Title of the Table><Corresponding Row Header><Corresponding Column Header>=<Entry Value>###'

### Check 3 formats of failed GPT-output and Hallucination:
#### 1. Failed Output Format 1 by check "sorry" or "apologize" like: I'm , ... I cannot generate ... Please provide ...
#### 2. Failed Output Format 2, Complete Output like: ### <...><...><...><...><...>=<...> ###, the Number of ">###" should be equal to "<###"
#### 3. Failed Output Format 3, Complete Output like: ### <...><...><...><...><...>=<...> ###, Number of ">###" should be equal to "<###" and ">=<"
### Check Hallucination: 

In [15]:
"""###Each entry value in GPT output ### <...><...><...><...><...>=<Entry Value>### has a corresponding value in original XML/HTML format <entry:entry value> in customized table"""

'###Each entry value in GPT output ### <...><...><...><...><...>=<Entry Value>### has a corresponding value in original XML/HTML format <entry:entry value> in customized table'

## Check Failed GPT-Output

In [17]:
import os
import shutil
import time

input_path = r'Data/GPT-CheckOut/InputTable' # Directory containing original customized table data in Step Two
output_path = r'Data/GPT-CheckOut/InputTable' # Directory containing GPT-Processed table data in Step 3.1
Fail_path = r'Data/GPT-CheckOut/Failed-Input' # Faield original customized table data in Step Two
Fail_path2 = r'Data/GPT-CheckOut/Failed-Output' # Failed GPT-Processed table data in Step 3.1

### Check Failed Output Format 1, like: I'm sorry, ... I cannot generate ... Please provide ...
def check_sorry():
    for file in os.listdir(output_path):
        if ".DS_Store" in file:
            continue

        file_path = os.path.join(output_path, file)
        if not os.path.exists(file_path):
            continue
        
        source_path = os.path.join(input_path, file)  # Moved here to avoid repeated declarations
        if not os.path.exists(source_path):
            continue

        des_path = None
        des_path2 = None

        try:
            with open(file_path, "r", encoding="utf-8") as f:
                file_content = f.read()

                if "sorry" or "apologize" in file_content:
                    print(f"'Sorry' found in {file_path}.")
                    des_path = os.path.join(Fail_path, file)
                    des_path2 = os.path.join(Fail_path2, file)
                    # Move source file to fail path
                    f.close()
                    shutil.move(source_path, des_path)
                    # Move the output file as well
                    shutil.move(file_path, des_path2)

                else:
                    ### Check Failed Output Format 2, Complete Output like: ### <...><...><...><...><...>=<...> ###, Number of "###" is even
                    nums10 = file_content.count(">###") + file_content.count("> ###")
                    nums20 = file_content.count("### <") + file_content.count("###<")
                    if nums10 != nums20:
                        print(f"Incomplete Input2 in {file_path}.")
                        des_path = os.path.join(Fail_path, file)
                        des_path2 = os.path.join(Fail_path2, file)
                        f.close()
                        shutil.move(source_path, des_path)
                        shutil.move(file_path, des_path2)
                    else:
                        ## Check Failed Output Format 3, Complete Output like: ### <...><...><...><...><...>=<...> ###, Number of "###" is twice number of ">=<"
                        nums1 = file_content.count(">=<")
                        if nums20 != nums1:
                            print(f"Incomplete Input3 in {file_path}.")
                            des_path = os.path.join(Fail_path, file)
                            des_path2 = os.path.join(Fail_path2, file)
                            f.close()
                            shutil.move(source_path, des_path)
                            shutil.move(file_path, des_path2)

        except FileNotFoundError as e:
            print(f"FileNotFoundError: {e} - Creating necessary directories and retrying.")
            # Ensure destination directory exists and retry
            os.makedirs(os.path.dirname(des_path), exist_ok=True)
            shutil.move(file_path, des_path)

        finally:
            # In case of issues, ensure we have proper paths before attempting move
            if des_path2 and os.path.exists(source_path):
                shutil.move(source_path, des_path2)

if __name__ == '__main__':
    for file in os.listdir(output_path):
        print(file)
        if ".DS_Store" in file:
            continue
        check_sorry()

## Check Hallucination

In [22]:
import os
import re
import shutil
import time

input_path = r'Data/GPT-CheckOut/InputTable' # Directory containing original customized table data in Step Two
output_path = r'Data/GPT-CheckOut/InputTable' # Directory containing GPT-Processed table data in Step 3.1
hallucination_path = r'Data/GPT-CheckOut/Failed-Input' # Faield original customized table data in Step Two (Hallucination)
hallucination_path2 = r'Data/GPT-CheckOut/Failed-Output' # Failed GPT-Processed table data in Step 3.1 (Hallucination2)

def check_hallucination():
    for file in os.listdir(output_path):
        if ".DS_Store" in file:
            continue
        file_path = os.path.join(output_path, file)
        inputjson_path = os.path.join(input_path, file)

        if not os.path.exists(inputjson_path):
            continue

        if not os.path.exists(file_path):
            continue
        with open(file_path, "r", encoding="utf-8") as f:
            file_content = f.read()
            # pattern = r">=<(\S+)>###"
            pattern = r"\B=<([^>]+>###"
            output_result = re.findall(pattern, file_content)
            print(output_result)
        
        if not os.path.exists(inputjson_path):
            shutil.move(inputjson_path, des_path)
            continue
        
        with open(inputjson_path, "r", encoding="utf-8") as f:
            file_content = f.read()
            pattern2 = r"<td:(.*?)>|<td\s+[^=]+=[^:]+:(.*?)>|<entry:(.*?)>|<entry\s+[^=]+=[^:]+:(.*?)>"
            input_result = re.findall(pattern2, file_content)
            all_result = []
            for i in input_result:
                all_result+=list(i)
            all_result = [i for i in all_result if i != ""]
            print(all_result)
            f.close()
        # Number Elements in input file
        # Delete Duplications
        output_filter = list(set(output_result))
        # print(output_filter)

        flag = False
        for filter in output_filter:
            output_num = output_result.count(filter)
            if filter == '-'or filter == '–' or filter == '—':
                output_num = output_result.count('-') + output_result.count('–') + output_result.count('—')
                input_num = all_result.count('-') + all_result.count('–') + all_result.count('—')
            else:
                input_num = all_result.count(filter)
            #print("filter:",filter)
            #print("output_num:",output_num)
            #print("input_num:",input_num)

            if input_num>=output_num and output_num>0:
            #    print("No Hallucination")
                pass
            else:
            #    print("Hallucination Exist")
                flag = True
                break
            # break
        if flag:
            # Hallucination
            print(f"Hallucination in：{file}")
            # sorry_path = os.path.join(Hallucination_path, file)
            if not os.path.exists(os.path.join(hallucination_path)):
                    os.makedirs(os.path.join(hallucination_path))
            des_path = os.path.join(hallucination_path, file)
            des_path2 = os.path.join(hallucination_path2, file)
            print(f"Source path: {inputjson_path}")
            print(f"Destination path 1: {des_path}")
            print(f"Destination path 2: {des_path2}")
            
            if not os.path.exists(os.path.dirname(des_path2)):
                os.makedirs(os.path.dirname(des_path2))
        
            try:
                shutil.move(inputjson_path, des_path)
                shutil.move(file_path, des_path2)
                
            except FileNotFoundError as e:
                print(f"Error moving file: {e}")
                print(f"Source file path: {file_path}")
                print(f"Destination file path: {des_path2}")
        else:
            print(f"No Hallucination in：{file}")

if __name__ == '__main__':
    # check_hallucination()  
    for file in os.listdir(output_path):
        if ".DS_Store" in file:
            continue
        table_path = os.path.join(output_path,file)
        check_hallucination()